In [4]:
dp1 = {'partno': 100, 'maxtemp': 35, 'mintemp': 35, 'maxvibration': 12, 'asperity': 0.32}
dp2 = {'partno': 101, 'maxtemp': 46, 'mintemp': 35, 'maxvibration': 21, 'asperity': 0.34}
dp3 = {'partno': 130, 'maxtemp': 56, 'mintemp': 46, 'maxvibration': 3412, 'asperity': 12.42}
dp4 = {'partno': 131, 'maxtemp': 58, 'mintemp': 48, 'maxvibration': 3542, 'asperity': 13.43}

It predicts the asperity according the maxvibration attribute.

In [5]:
def predict(dp):
    if(dp['maxvibration'] > 100):
        return 13
    else:
        return 0.33

In [15]:
predict(dp1)

0.33

In [8]:
predict(dp2)

0.33

In [10]:
predict(dp3)

13

In [11]:
predict(dp4)

13

In [12]:
w1 = 0.30
w2 = 0
w3 = 0
w4 = 13 / 3412.0

def mlpredict(dp):
    return w1 + w2*dp['maxtemp'] + w3 * dp['mintemp'] + w4 * dp['maxvibration']

In [13]:
mlpredict(dp4)

13.795310668229778

In [14]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [16]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2020-06-26 11:11:57--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2020-06-26 11:11:58--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2020-06-26 11:11:59--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.0.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (

In [17]:
df_energy = spark.sql("""
select sqrt(sum(x * x + y * y + z * z)) as ablel, class from df group by class
""").show()

+------------------+--------------+
|             ablel|         class|
+------------------+--------------+
| 8959.680239829991| Use_telephone|
| 9737.511232342687| Standup_chair|
| 12542.96539897962|      Eat_meat|
|13225.945637269193|     Getup_bed|
|15003.269043778426|   Drink_glass|
|14454.885091207056|    Pour_water|
|10616.408809008817|     Comb_hair|
|16537.370891408344|          Walk|
|11082.626493751379|  Climb_stairs|
|10261.338314274606| Sitdown_chair|
|6783.4063714331605|   Liedown_bed|
| 7173.493500380411|Descend_stairs|
| 11785.39634462923|   Brush_teeth|
| 6071.460120926432|      Eat_soup|
+------------------+--------------+



In [18]:
dp1 = {'partno': 100, 'maxtemp': 35, 'mintemp': 35, 'maxvibration': 12, 'broken': 0}
dp2 = {'partno': 101, 'maxtemp': 46, 'mintemp': 35, 'maxvibration': 21, 'broken': 0}
dp3 = {'partno': 130, 'maxtemp': 56, 'mintemp': 46, 'maxvibration': 3412, 'broken': 1}
dp4 = {'partno': 131, 'maxtemp': 58, 'mintemp': 48, 'maxvibration': 3542, 'broken': 1}

In [19]:
def predict(dp):
    if dp['maxvibration'] > 100:
        return 1
    else:
        return 0

In [20]:
predict(dp1)

0

In [21]:
predict(dp2)

0

In [22]:
predict(dp3)

1

In [23]:
predict(dp4)

1

In [44]:
import math

def sigmoid(x):
    return 1/(1+math.exp(-x))

In [50]:
w1 = 0.30
w2 = 0
w3 = 0
w4 = 13 / 3412.0

def mlpredict(dp):
    return 1 if sigmoid(w1 + w2*dp['maxtemp'] + w3 * dp['mintemp'] + w4 * dp['maxvibration']) > 0.7 else 0

In [51]:
mlpredict(dp1)

0

In [52]:
mlpredict(dp2)

0

In [53]:
mlpredict(dp3)

1

In [54]:
mlpredict(dp4)

1